In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5500")
mlflow.set_experiment("rag")
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

/home/mike/git/dspy-tool-use/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dspy
import openai
import os

LLM_URL=os.getenv('LLM_URL', 'http://localhost:8080/v1')
API_KEY=os.getenv('API_KEY', 'fake')
LLM_MODEL=os.getenv('LLM_MODEL', 'openai/Llama-3.2-3B-Instruct-Q8_0.gguf')
MAX_TOKENS=os.getenv('MAX_TOKENS', 6000)
TEMPERATURE=os.getenv('TEMPERATURE', 0.2)
dspy.enable_logging()
lm = dspy.LM(model=LLM_MODEL,
             api_base=LLM_URL,  # ensure this points to your port
             api_key=API_KEY,
             temperature=TEMPERATURE,
             model_type='chat',
             stream=False)
dspy.configure(lm=lm)
#dspy.settings.configure(track_usage=True)

In [ ]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

10:37:44 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
10:37:47 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
10:37:47 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
10:37:47 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
10:37:47 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: Ll

Trace(request_id=d25a6d2ee5694ab4a14fc5f36003b094)

10:40:01 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
10:40:01 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: Llama-3.2-3B-Instruct-Q8_0.gguf
10:40:01 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
10:40:01 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: Llama-3.2-3B-Instruct-Q8_0.gguf
10:42:04 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gg

In [9]:
dspy.inspect_history(n=1)





[2025-06-03T10:37:47.458714]

System message:

Your input fields are:
1. `question` (str)
Your output fields are:
1. `response` (str)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
what are high memory and low memory on linux?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## response ## ]]
High memory and low memory are two terms used to describe the amount of free memory available on a Linux system. High memory refers to a system with a significant amount of free memory, typically above 50% of the total system memory. This allows for smooth performance, as the system

In [3]:
cot = dspy.ChainOfThought('question -> response')
cot(question="what is red hat linux?")

Prediction(
    reasoning='Red Hat Linux is a popular open-source operating system that is widely used in servers, desktops, and mobile devices. It is based on the Linux kernel and is known for its stability, security, and ease of use. Red Hat Linux is also a commercial version of Linux, which means it is supported by Red Hat, a company that provides free and paid support, as well as a wide range of software packages and services.',
    response='Red Hat Linux is a popular open-source operating system that is widely used in servers, desktops, and mobile devices. It is based on the Linux kernel and is known for its stability, security, and ease of use. Red Hat Linux is also a commercial version of Linux, which means it is supported by Red Hat, a company that provides free and paid support, as well as a wide range of software packages and services.'
)

Trace(request_id=f1c8af4192c14196a201858d45625f7e)

In [4]:
import ujson
from dspy.utils import download

# Download question--answer pairs from the RAG-QA Arena "Tech" dataset.
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl")

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

In [5]:
# Inspect one datapoint.
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

In [6]:
data = [dspy.Example(**d).with_inputs('question') for d in data]

# Let's pick an `example` here from the data.
example = data[2]
example

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

In [7]:
import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)

(200, 300, 500)

In [8]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

print(example)
print(pred)

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})
Prediction(
    reasoning='Your text messages are coming up as "maybe" because of the way you\'re formatting your text messages. In many messaging platforms, including SMS and iMessage, the default formatting for text messages is to use a "maybe" or "unknown" sender ID if the phone number is not recognized or if the message is sent to a number that is not in the recipient\'s contact list.\n\nThis is a security feature designed to prevent spam messages f

[Trace(request_id=2387788d4b114f75ae0bfc0e0114cfc4), Trace(request_id=f54cb3565b1e4aca9fd93d291329a941)]

In [9]:
dspy.inspect_history(n=1)





[2025-06-03T11:10:59.081299]

System message:

Your input fields are:
1. `question` (str)
2. `ground_truth` (str)
3. `system_response` (str)
Your output fields are:
1. `reasoning` (str)
2. `ground_truth_key_ideas` (str): enumeration of key ideas in the ground truth
3. `system_response_key_ideas` (str): enumeration of key ideas in the system response
4. `discussion` (str): discussion of the overlap between ground truth and system response
5. `recall` (float): fraction (out of 1.0) of ground truth covered by the system response
6. `precision` (float): fraction (out of 1.0) of system response covered by the ground truth
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## ground_truth ## ]]
{ground_truth}

[[ ## system_response ## ]]
{system_response}

[[ ## reasoning ## ]]
{reasoning}

[[ ## ground_truth_key_ideas ## ]]
{ground_truth_key_ideas}

[[ ## system_response_key_ideas ## ]]
{system_response_k

In [ ]:
import mlflow

with mlflow.start_run(run_name="rag_evaluation"):
    evaluate = dspy.Evaluate(
        devset=devset,
        metric=metric,
        num_threads=24,
        display_progress=True,
        # To record the outputs and detailed scores to MLflow
        return_all_scores=True,
        return_outputs=True,
    )

    # Evaluate the program as usual
    aggregated_score, outputs, all_scores = evaluate(cot)


    # Log the aggregated score
    mlflow.log_metric("semantic_f1_score", aggregated_score)
    # Log the detailed evaluation results as a table
    mlflow.log_table(
        {
            "Question": [example.question for example in eval_set],
            "Gold Response": [example.response for example in eval_set],
            "Predicted Response": outputs,
            "Semantic F1 Score": all_scores,
        },
        artifact_file="eval_results.json",
    )

  0%|          | 0/300 [00:00<?, ?it/s]

11:11:17 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:11:17 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:11:17 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:11:17 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:11:17 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM c

Average Metric: 0.67 / 1 (66.7%):   0%|          | 1/300 [00:53<4:26:15, 53.43s/it]

11:12:10 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:12:15 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:12:15 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:12:15 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:12:15 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: op

Average Metric: 1.28 / 2 (64.1%):   1%|          | 2/300 [00:58<2:02:46, 24.72s/it]

...

11:19:28 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:19:30 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:19:30 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:19:30 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:19:30 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: op

Average Metric: 82.18 / 145 (56.7%):  49%|████▊     | 146/300 [08:15<08:28,  3.30s/it]

11:19:32 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:19:33 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:19:33 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:19:33 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:19:33 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: op

Average Metric: 82.85 / 146 (56.7%):  49%|████▉     | 147/300 [08:20<09:22,  3.68s/it]

11:19:37 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:19:39 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:19:39 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:19:39 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:19:39 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: op

In [ ]:
# Define an evaluator that we can re-use.
#evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24,
#                         display_progress=True, display_table=2)

# Evaluate the Chain-of-Thought program.
#evaluate(cot)

  0%|          | 0/300 [00:00<?, ?it/s]

11:07:07 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:07:07 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:07:07 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:07:07 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
11:07:07 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM c

Average Metric: 0.00 / 1 (0.0%):   0%|          | 1/300 [00:46<3:51:57, 46.55s/it]

11:07:53 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:07:54 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:07:54 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:07:54 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:07:54 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: op

Average Metric: 0.67 / 2 (33.3%):   1%|          | 2/300 [00:47<1:38:20, 19.80s/it]

...

11:08:29 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:08:29 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:08:29 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:08:29 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:08:29 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= Llama-3.2-3B-Instruct-Q8

🏃 View run eval at: http://localhost:5500/#/experiments/414799578984116612/runs/6aadb11566024c76b4a1aae148c9766a
🧪 View experiment at: http://localhost:5500/#/experiments/414799578984116612


KeyboardInterrupt: 

[Trace(request_id=53971cb679ba436ba6a325692933b7c8), Trace(request_id=0287c32ebfd94dc4b936b95e75b22825), Trace(request_id=0d0a21fc052f48548a8c6eb99f4212da), Trace(request_id=bef4fa60901549688fee28659edeb8ce), Trace(request_id=3ec807d87f3c49afa40249587145c505), Trace(request_id=6db5cbd8eac7457db5faa855a694f958), Trace(request_id=811a69b58dd648cd971de13d33ca67a5), Trace(request_id=49912767847740babca64d63a539c480), Trace(request_id=81ccbab23e9f4fd49e363b5d5a60a916), Trace(request_id=906f01f267f941c3b13d2e3f61c586d3)]

INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:08:43 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:08:43 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
11:08:43 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
11:08:44 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
11:08:44 - LiteLLM:INFO: cost_calculator.py:655 - selected model n